In [33]:


import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import multivariate_normal
import random as rn
#import eif as iso
import seaborn as sb
sb.set_style(style="whitegrid")
sb.set_color_codes()
import scipy.ndimage
from scipy.interpolate import griddata
import numpy.ma as ma
from numpy.random import uniform, seed
import pandas as pd
import csv
from google.colab import drive
from scipy import stats
import datetime
dir=!ls
print(dir[0])
drive.mount("/content/gdrive")

datasets.py  DECRuL.ipynb  metrics.py	README.md
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Load Modules**

In [59]:
%load_ext autoreload
%autoreload 2
import os
os.chdir('/content/gdrive/My Drive/research/AnomalyDetection/AnomalyDetection')
import DEC as DEC
from DEC import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
# load normal points before applied isolation forest

import requests
from io import StringIO

orig_url='https://drive.google.com/file/d/1D9HS-ZD3VKzu_bxhJTiJdWi3ZjrNOcAc/view?usp=sharing'

file_id = orig_url.split('/')[-2]
dwn_url='https://drive.google.com/uc?export=download&id=' + file_id
url = requests.get(dwn_url).text
csv_raw = StringIO(url)
dfs_rul = pd.read_csv(csv_raw)


In [38]:
dfs_rul[1:-1]

,id,CROWD_EXTENSION__c3__lag_4,CROWD_SPEED_REF_INV__cid_ce__normalize_False,CROWD_SPEED_REF_INV__number_peaks__n_2,DIPEER_TRIP__standard_deviation,HOIST_IW_INV__absolute_sum_of_changes,HOIST_IW_INV__cid_ce__normalize_False,HOIST_KW__absolute_sum_of_changes,HOIST_KW__cid_ce__normalize_False,HOIST_ROPE_LENGTH_INV__c3__lag_4,HOIST_SPEED_INV__absolute_sum_of_changes,HOIST_SPEED_INV__cid_ce__normalize_False,HOIST_SPEED_REF_INV__absolute_sum_of_changes,HOIST_SPEED_REF_INV__cid_ce__normalize_False,HOIST_TORQUE_INV__absolute_sum_of_changes,HOIST_TORQUE_INV__cid_ce__normalize_False,SWING_ANGLE__c3__lag_4
1,2577,0.690549,0.369734,0.133333,0.563823,0.215079,0.420271,0.284966,0.450589,0.878978,0.319497,0.511453,0.301992,0.584629,0.222194,0.343218,0.391754
2,2610,0.651425,0.431057,0.133333,0.557000,0.223752,0.402818,0.306012,0.486778,0.889080,0.314528,0.510403,0.285598,0.582648,0.207818,0.287451,0.412539
3,284,0.621482,0.458988,0.133333,0.570899,0.181160,0.383905,0.289601,0.482390,0.873782,0.352894,0.541146,0.348457,0.611439,0.217654,0.359512,0.407546
4,171,0.611547,0.426934,0.133333,0.537914,0.227400,0.444468,0.297195,0.533427,0.910510,0.322822,0.511552,0.303909,0.541235,0.241362,0.318816,0.412187
5,63,0.669858,0.411247,0.133333,0.557000,0.212962,0.418730,0.317214,0.519267,0.905903,0.318438,0.484861,0.330962,0.578603,0.269357,0.405230,0.388985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1847,1636,0.611288,0.416431,0.000000,0.578245,0.321895,0.642811,0.343544,0.569296,0.902573,0.270535,0.427030,0.293251,0.590049,0.334931,0.614291,0.320218
1848,2665,0.729318,0.484197,0.400000,0.446672,0.260614,0.439573,0.356543,0.494488,0.939142,0.438176,0.592257,0.407217,0.676809,0.323329,0.386944,0.449694
1849,983,0.794959,0.307458,0.066667,0.737564,0.203239,0.533226,0.219488,0.428225,0.820516,0.208090,0.426219,0.215634,0.565564,0.217907,0.494966,0.505991
1850,1397,0.652139,0.327061,0.133333,0.806226,0.128724,0.417926,0.273537,0.654321,0.824642,0.213680,0.540468,0.229025,0.603390,0.115511,0.240244,0.521940


In [39]:

x=dfs_rul.drop(['id'],axis=1)

In [40]:
x.head()
x.shape

(1853, 16)

In [41]:
pretrain_optimizer = SGD(lr=1, momentum=0.9)
pretrain_epochs = 20
batch_size=256
n_clusters = 6
save_dir='results'
init = 'glorot_uniform'

In [42]:
# prepare the DEC model
dec = DEC(dims=[x.shape[-1], 500, 500, 2000,6], n_clusters=n_clusters, init=init)

6


In [43]:
dec.pretrain(x=x, y=None, optimizer=pretrain_optimizer,
                     epochs=pretrain_epochs, batch_size=batch_size,
                     save_dir=save_dir)

...Pretraining...
Epoch 1/20
8/8 [==============================] - 0s 4ms/step - loss: 0.0764
Epoch 2/20
8/8 [==============================] - 0s 4ms/step - loss: 0.0137
Epoch 3/20
8/8 [==============================] - 0s 5ms/step - loss: 0.0077
Epoch 4/20
8/8 [==============================] - 0s 4ms/step - loss: 0.0065
Epoch 5/20
8/8 [==============================] - 0s 4ms/step - loss: 0.0059
Epoch 6/20
8/8 [==============================] - 0s 4ms/step - loss: 0.0058
Epoch 7/20
8/8 [==============================] - 0s 4ms/step - loss: 0.0056
Epoch 8/20
8/8 [==============================] - 0s 4ms/step - loss: 0.0056
Epoch 9/20
8/8 [==============================] - 0s 4ms/step - loss: 0.0055
Epoch 10/20
8/8 [==============================] - 0s 3ms/step - loss: 0.0055
Epoch 11/20
8/8 [==============================] - 0s 3ms/step - loss: 0.0054
Epoch 12/20
8/8 [==============================] - 0s 3ms/step - loss: 0.0053
Epoch 13/20
8/8 [==============================] - 0s 3

In [44]:
dec.model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 16)]              0         
_________________________________________________________________
encoder_0 (Dense)            (None, 500)               8500      
_________________________________________________________________
encoder_1 (Dense)            (None, 500)               250500    
_________________________________________________________________
encoder_2 (Dense)            (None, 2000)              1002000   
_________________________________________________________________
encoder_3 (Dense)            (None, 6)                 12006     
_________________________________________________________________
clustering (ClusteringLayer) (None, 6)                 36        
Total params: 1,273,042
Trainable params: 1,273,042
Non-trainable params: 0
____________________________________________

In [45]:
dec.compile(optimizer=SGD(0.01, 0.9), loss='kld')

In [46]:
q = dec.model.predict(x, verbose=0)

In [47]:
q

array([[0.15709466, 0.19424242, 0.15040964, 0.14881755, 0.16028416,
        0.1891516 ],
       [0.1587127 , 0.197363  , 0.15198857, 0.14788105, 0.16073194,
        0.18332274],
       [0.15962027, 0.19779019, 0.15267164, 0.14705497, 0.16189267,
        0.18097024],
       ...,
       [0.16201694, 0.2013835 , 0.15447737, 0.14516225, 0.16268949,
        0.17427051],
       [0.15316814, 0.19001132, 0.14761883, 0.15195   , 0.15780126,
        0.19945045],
       [0.16703013, 0.20492993, 0.1576853 , 0.14155586, 0.16632654,
        0.16247226]], dtype=float32)

In [48]:
q.argmax(1)

array([1, 1, 1, ..., 1, 5, 1])

In [49]:
q.sum(0)

array([291.79562, 360.83826, 279.24933, 275.8948 , 297.00482, 348.2176 ],
      dtype=float32)

In [17]:
# load dataset


In [192]:
%autoreload 2

In [194]:

y_pred = dec.fit(x, y=None, tol=0.001, maxiter=100, batch_size=batch_size,
                     update_interval=140, save_dir=save_dir)
    #print('acc:', metrics.acc(y, y_pred))
   # print('clustering time: ', (time() - t0))

Update interval 140
Save interval 35
Initializing cluster centers with k-means:::
6
(1853, 6)
[4 4 3 4 4 4 4 4 0 0 3 3 0 4 4 0 4 0 3 4]
debug begin
debug end-1
debug end--2
p [[0.17275287 0.16227286 0.15372875 0.17075697 0.17394677 0.1665418 ]
 [0.17045209 0.17206134 0.14460835 0.17657842 0.17582537 0.1604744 ]
 [0.16908118 0.17745642 0.13995895 0.17962065 0.17664851 0.15723425]
 ...
 [0.16467759 0.19313219 0.1281528  0.18757851 0.17812102 0.1483379 ]
 [0.1765404  0.1389716  0.1807663  0.15487579 0.16688487 0.18196103]
 [0.15800884 0.21496263 0.11450446 0.19697493 0.178503   0.13704616]]
debuf end---3
0.0
debug end--4
debug end--5
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88 

In [196]:
#print('clustering time: ', (time() - t0))